# CS 105 Team 7 Mini Project Report

[]

[]

1. What data do you have? (Your answer)
2. What would you like to know? (Your answer)
3. Explore, present and analyze the data. (Generate statistics, perform visualizations)
4. Can you state any hypotheses or make predictions? Which tests can you apply to
5. Verify your hypothesis? (Your answer).
6. Test your hypotheses. Draw the conclusions.
7. Draw the overall conclusion



In [ ]:
#test 1, lets cook

In [ ]:
#Lydia's Chunk: Heatmap Visualization
# Comparing the on campus choice of food vs scale of imposter syndrome (1-5)
# On campus food choice: EV - FE
# Imposter syndrome: DM